In [8]:
!pip install selenium html5lib lxml

In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import hashlib
import time

In [8]:
driver = webdriver.Edge("C:\\Users\\lnb\\scoop\\apps\\edgedriver\\current\\msedgedriver.exe")
date = "20210321"
gameId = date + "HTSS02021"
url = "https://sports.news.naver.com/gameCenter/textRelay.nhn?category=kbo&gameId=" + gameId

driver.implicitly_wait(5)

In [22]:
def save_lineup_to_csv(_lineup_box):
    lineup_title_bar = _lineup_box.find_element_by_xpath('//*[@id="lineup_box"]/h3')
    lineup_title = lineup_title_bar.get_attribute('innerText').split(' ', 1)[0]
    # 투수 라인업
    tusu_table = _lineup_box.find_element_by_xpath('//*[@id="lineup_box"]/div/table[1]')
    # 타자 라인업
    taja_table = _lineup_box.find_element_by_xpath('//*[@id="lineup_box"]/div/table[2]')

    tusu_df = pd.read_html(tusu_table.get_attribute('outerHTML'))[0]
    tusu_df["이름"].to_csv("./csv/" + date + "-" + lineup_title + "-투수.csv", index=False, encoding="utf-8-sig", index_label=None, header=False)

    taja_df = pd.read_html(taja_table.get_attribute('outerHTML'))[0]
    taja_df["이름"].to_csv("./csv/" + date + "-" + lineup_title + "-타자.csv", index=False, encoding="utf-8-sig", index_label=None, header=False)

    # div 없애기
    lineup_title_bar.click()



driver.get(url)

away_lineup_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="away_lineup_btn"]'))
)
home_lineup_btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="home_lineup_btn"]'))
)

away_lineup_btn.click()
lineup_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="lineup_box"]'))
)
save_lineup_to_csv(lineup_box)

home_lineup_btn.click()
lineup_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="lineup_box"]'))
)
save_lineup_to_csv(lineup_box)

In [ ]:
current_player_text_hash = ""
current_player_text = ""

entire_game_text_hash = ""
entire_game_text = ""

while 1:
    driver.get(url)

    entire_innings_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "inning_tab_all"))
    )

    entire_innings_button.click()

    current_player_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "current_player_text"))
    )

    textbox = driver.find_element_by_xpath('//*[@id="relay_text"]')
    entire_game_text = textbox.get_attribute("innerText")
    if "경기종료" in entire_game_text:
        print("경기가 종료되었습니다")
        break

    enc = hashlib.md5()
    current_player_text = current_player_element.get_attribute("innerText")

    enc.update(current_player_text.encode('utf-8'))
    if current_player_text_hash != enc.hexdigest():
        print("===== 현재 선수 상황 =====")
        print(current_player_text.replace("\n\n", "\n"))
        current_player_text_hash = enc.hexdigest()

    enc.update(entire_game_text.encode('utf-8'))
    if entire_game_text_hash != enc.hexdigest():
        print("===== 경기 전체 상황 =====")
        # print(entire_game_text.replace("\n\n", "\n"))
        print(entire_game_text_hash)
        entire_game_text_hash = enc.hexdigest()

    time.sleep(5)